In [1]:
import os
os.chdir('..')
os.chdir('..')

In [2]:
from tqdm import tqdm

import numpy as np
import pandas as pd
# from scipy.stats import t

from bokeh.plotting import show
# from bokeh.models.annotations import Title
#from bokeh.models import Plot, ColumnDataSource, Ellipse, Grid, LinearAxis, Text, Scatter
from bokeh.io import output_notebook

from puncta_counter.utils.common import title_to_snake_case
from puncta_counter.utils.ellipse_algos import min_vol_ellipse
from puncta_counter.utils.plotting import (save_plot_as_png,
                                           instantiate_plot_bokeh,
                                           plot_ellipse_using_bokeh,
                                           plot_scatter_using_bokeh)

pd.options.display.max_columns = None
output_notebook()

Loading BokehJS ...

In [3]:
save=True

In [4]:
def min_vol_ellipse(P, tolerance=0.05, **kwargs):
    """See: https://www.mathworks.com/matlabcentral/fileexchange/9542-minimum-volume-enclosing-ellipsoid
    Original author: Nima Moshtagh (nima@seas.upenn.edu)
    Translated to Python by Harrison Wang
    There's a bug in the current implementation
    """

    # Data Points
    # -----------------------------------
    
    d, N = P.shape
    if N <= d:
        return np.nan, np.nan, np.nan, np.nan, np.nan
    
    # Q = np.zeros((d+1,N))
    # Q(1:d,:) = P(1:d,1:N)
    # Q(d+1,:) = np.ones(1,N)
    Q = np.vstack([P, np.ones((1, N))])


    # Initializations
    # -----------------------------------
    count = 1
    err = 1
    u = (1/N) * np.ones((N, 1))  # 1st iteration
    

    # Khachiyan Algorithm
    # -----------------------------------
    while err > tolerance:
        try:
            X = np.dot(np.dot(Q, u* np.identity(N)), np.transpose(Q))
            M = np.diag(np.dot(np.dot(np.transpose(Q), np.linalg.inv(X)), Q))  # M the np.diagonal vector of an NxN matrix

            j = np.argmax(M)
            maximum = max(M)

            step_size = (maximum - d -1)/((d+1)*(maximum-1))
            new_u = (1 - step_size)*u 
            new_u[j] = new_u[j] + step_size

            count = count + 1
            err = np.linalg.norm(new_u - u)
            u = new_u
        except:
            print(P)
            break


    ################### Computing the Ellipse parameters######################
    # Finds the ellipse equation in the 'center form': 
    # (x-c)' * A * (x-c) = 1
    # It computes a dxd matrix 'A' and a d dimensional vector 'c' as the center
    # of the ellipse. 
    U = u * np.identity(N)
    # return P, u
    
    # the A matrix for the ellipse
    # A = (1/d) * inv(P * U * P' - (P * u)*(P*u)' );
    # --------------------------------------------
    A = (1/d) * np.linalg.inv(
        np.dot(np.dot(P, U), np.transpose(P)) - np.dot(np.dot(P, u), np.transpose(np.dot(P, u)))
    )
    
    # center of the ellipse 
    # --------------------------------------------
    c = np.dot(P, u)
    
    # original return value
    return A, c

    center_x = c[0][0]
    center_y = c[1][0]
    
    eig_vals, eig_vecs = np.linalg.eig(A)

    minor_axis_length, major_axis_length = sorted([2/np.sqrt(eig_vals[0]), 2/np.sqrt(eig_vals[1])])
    orientation = np.arcsin(eig_vecs[0, 1])/np.pi*180
    
    return center_x, center_y, major_axis_length, minor_axis_length, orientation

# Test Algo

In [5]:
x = np.array([425.        , 423.        , 381.        , 384.        ,
        382.        , 384.        , 386.8       , 390.85      ,
        394.51851852, 397.85714286, 387.77777778, 389.7       ,
        392.61538462, 397.45454545, 386.        , 396.59259259,
        401.        , 402.        ])[2:]
x = x-np.mean(x)
y = np.array([842.        , 848.        , 881.        , 882.        ,
        884.66666667, 885.5       , 886.7       , 887.2       ,
        887.03703704, 887.57142857, 888.88888889, 891.3       ,
        892.53846154, 890.45454545, 893.        , 893.88888889,
        893.        , 895.5       ])[2:]
y = y-np.mean(y)

In [6]:
A, c = min_vol_ellipse(np.array((x, y)), tolerance=0.001)

In [7]:
center_x = c[0][0]
center_y = c[1][0]

eig_vals, eig_vecs = np.linalg.eig(A)

minor_axis_length, major_axis_length = sorted([2/np.sqrt(eig_vals[0]), 2/np.sqrt(eig_vals[1])])
orientation = np.arcsin(eig_vecs[0, 1])/np.pi*180

In [8]:
np.linalg.eig(A)

(array([0.00559143, 0.03210527]),
 array([[-0.87165586,  0.49011842],
        [-0.49011842, -0.87165586]]))

In [9]:
orientation

29.34836535288394

In [ ]:
title = 'Minimum Volume Ellipsoid Algorithm'
title_f = title_to_snake_case(title)

plot = instantiate_plot_bokeh(
    width=600,
    height=400,
    x_range=[-15, 15],
    y_range=[-10, 10],)
plot = plot_scatter_using_bokeh(
    pd.DataFrame({'x':x , 'y': y}),
    plot=plot
)
plot = plot_ellipse_using_bokeh(
    pd.DataFrame(
        {'x': [c[0][0]] , 'y': [c[1][0]],
         'height': [minor_axis_length],
         'width': [major_axis_length],
         'angle': [orientation]
    }),
    fill_alpha=0.2,
    title=title,
    plot=plot,
)

if save:
    save_plot_as_png(plot, f"figures/examples/{title_f}.png")

show(plot)